In [3]:
from openai import OpenAI
from apikey import api_key
client = OpenAI(api_key=api_key)

In [4]:
def readfile(path):
    with open(path, 'r') as file:
        return file.read()
docu =readfile("envision-brief.md")    
challenge = readfile("mychallenges/c"+"008"+".md")

# decompose challenge into question and prof_answer
def decompose_challenge(challenge):
    question,prof_answer=challenge.split("\n\n# ANSWER\n\n")
    return question, prof_answer
question,prof_answer=decompose_challenge(challenge)
print(question,prof_answer)

Create a table Catalog containing different items and their color. "red" should be a color cited. Create another table containing the same dimension but only with the red items. Show the Labels of the red items. ```envision
table Catalog = with 
  [| as Label, as Color |]
  [| "Socks" , "red"    |]
  [| "Socks" , "blue"   |]
  [| "Shirt" , "red"    |] 
 
table smallCatalog = where Catalog.Color=="red" // broadcast during definition
 
show table "red items" with
  smallCatalog.Label
```


In [5]:
coder_personality="You are a proficient coder in the Domain Specific Language called Envision. \
    Your task is to generate response to the given challenge. \
    Some challenges will ask you to generate Envision code,\
    others will ask you to explain given code or answer questions related to the Envision language. \
    Do not output any intermediate thinking, only give the final answer.\
    Here is the documentation of Envision:\
    ### Documentation\n" + docu
coder_prompt=question

coder_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": coder_personality},
        {"role": "user", "content": coder_prompt}
    ],
    max_tokens=1000,  # Adjust the number of tokens based on your needs
    temperature=0.2,
)
stud_sentence=coder_response.choices[0].message.content
print(stud_sentence)


```envision
table Catalog = with
  [| as Item, as Color |]
  [| "Apple", "red" |]
  [| "Banana", "yellow" |]
  [| "Cherry", "red" |]
  [| "Grapes", "green" |]

table RedItems = where Catalog.Color == "red"

show table "Red Items" a1b3 with
  RedItems.Item
```


In [7]:

# this personality sticks more to professor's answer.

judge_personality_teacherAuthority="Your goal is to judge whether the STUDENT ANSWER is OK or NOT_OK, as an answer to the QUESTION.\
In order to judge the STUDENT ANSWER, you are given the PROFESSOR ANSWER which is always OK.\
You should judge NOT_OK if there is something UNACCEPTABLE in the STUDENT ANSWER; and judge OK if everything is ACCEPTABLE.\
Your main job is not to check the syntax correctness, but the logical correctness.\
If the STUDENT ANSWER does not treat the QUESTION logically, it is UNACCEPTABLE.\
Pay special attention to the comments in the PROFESSOR ANSWER. If these comments include\
a rule and if the STUDENT ANSWER violates it, this is UNACCEPTABLE.\
Adding a print position label like a1b2 in the show command is ACCEPTABLE, as long as this is not forbidden by\
the QUESTION or the comments in the PROFESSOR ANSWER.\
Differences in variable names, column names and table names shall systematically be ACCEPTABLE, \
as long as it is not specifically required in the QUESTION or in the comments of the PROFESSOR ANSWER.\
There are sometimes various ways or logics to treat the same QUESTION, and this is ACCEPTABLE, as long as the goal of the QUESTION is achieved.\
Let's think aloud step by step to help your judgement. Tell each ACCEPTABLE or UNACCEPTABLE point. \
At the end of your output, you must add a seperate line which ONLY contains your judgement as expressed in '0' for NOT_OK or '1' for OK,\
according to your previous conclusions, and NOTHING MORE!"

judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence

# Generate a response from the chatbot
judge_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": judge_personality_teacherAuthority},
        {"role": "user", "content": judge_prompt}
    ],
    max_tokens=800,  # Adjust the number of tokens based on your needs
    temperature=0.2,
)
print(judge_prompt)
# Print the generated response
print(judge_response.choices[0].message.content)


### QUESTION: Create a table Catalog containing different items and their color. "red" should be a color cited. Create another table containing the same dimension but only with the red items. Show the Labels of the red items.
### PROFESSOR ANSWER: ```envision
table Catalog = with 
  [| as Label, as Color |]
  [| "Socks" , "red"    |]
  [| "Socks" , "blue"   |]
  [| "Shirt" , "red"    |] 
 
table smallCatalog = where Catalog.Color=="red" // broadcast during definition
 
show table "red items" with
  smallCatalog.Label
```
### STUDENT ANSWER: ```envision
table Catalog = with
  [| as Item, as Color |]
  [| "Apple", "red" |]
  [| "Banana", "yellow" |]
  [| "Cherry", "red" |]
  [| "Grapes", "green" |]

table RedItems = where Catalog.Color == "red"

show table "Red Items" a1b3 with
  RedItems.Item
```
- The STUDENT ANSWER creates a table named Catalog with columns named Item and Color, which is different from the columns named Label and Color in the PROFESSOR ANSWER. This difference is ACCEP

In [8]:
# extract the 'real' code from the student answer (cut away the '''envision bit at the start and end)
def extract_code(stud_sentence):
    lines = stud_sentence.strip().split('\n')
    return '\n'.join(lines[1:-1])
print(extract_code(stud_sentence))

table Catalog = with
  [| as Item, as Color |]
  [| "Apple", "red" |]
  [| "Banana", "yellow" |]
  [| "Cherry", "red" |]
  [| "Grapes", "green" |]

table RedItems = where Catalog.Color == "red"

show table "Red Items" a1b3 with
  RedItems.Item


In [9]:
import requests
# send code to online compiler and check if it compiles

def check_compilation(script):
    url = "https://try.lokad.com/w/script/trycompile"
    payload = {
        "Script": script
    }

    try:
        # Send POST request
        response = requests.post(url, json=payload)

        # Check for successful response
        if response.status_code == 200:
            result = response.json()
            if result["IsCompOk"]:
                return True
            else:
                print("Compilation Failed!")
                for message in result["CompMessages"]:
                    print(f"Error: {message['Text']} (Line: {message['Line']}, Start: {message['Start']}, Length: {message['Length']}, Severity: {message['Severity']})")
                    return False
        else:
            print("Error: Unable to reach the compilation service.")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

        

# Example usage
check_compilation(extract_code(stud_sentence))


True

In [10]:
# for each question, try 3 generation-compilations.
# if compiles, further check with judge.
def pipeline_verify(challenge,coder_personality,judge_personality=judge_personality_teacherAuthority):
    question,prof_answer=decompose_challenge(challenge)
    n_tries=3
    for compile_try in range(n_tries):
        coder_prompt=question
        coder_response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": coder_personality},
                {"role": "user", "content": coder_prompt}
            ],
            max_tokens=1000,  # Adjust the number of tokens based on your needs
            temperature=0.2,
        )
        stud_sentence=coder_response.choices[0].message.content
        if(check_compilation(extract_code(stud_sentence))):
            print('# compile ok')
            break
        elif (compile_try==n_tries-1):
            print( "# too many failures !")
            print('# badcode:\n'+extract_code(stud_sentence))
            return stud_sentence,"too many failures !",False

    judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence
    judge_response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": judge_personality},
            {"role": "user", "content": judge_prompt}
        ],
        max_tokens=800,  # Adjust the number of tokens based on your needs
        temperature=0.2,
    )
    judge_sentence=judge_response.choices[0].message.content
    judge_decision=judge_sentence[-1]=='1'
    print ('# judge_decision:',judge_decision)
    if not judge_decision:
        print('# badcode:\n',extract_code(stud_sentence))
        print('# judge explanation:\n',judge_sentence)
    return stud_sentence,judge_sentence,judge_decision
# a all-in-one function to score a model on a list of challenges
def pipeline_score_allchallenge(indexes,coder_personality):
    challenges=[readfile("mychallenges/c"+index+".md") for index in indexes]
    score=0
    for i in range(len(challenges)):
        challenge=challenges[i]
        print('\n### verifying challenge No. '+indexes[i])
        _,_,judge_decision=pipeline_verify(challenge,coder_personality)
        if (judge_decision): score+=1
    print('correct:'+str(score)+' out of '+str(len(challenges))+', '+str(score/len(challenges)*100)+'%')  

In [12]:
# test the pipeline. typically fails on challenges 0,4,7 
# (these questions include grammar rules not covered in the documentation)

# do not use 005(which is not a question of code generation, does not fit in the pipeline format.)

indexes=["000","001","002","003","004","006","007","008"]
pipeline_score_allchallenge(indexes,coder_personality)


### verifying challenge No. 000
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 5, Start: 20, Length: 4, Severity: Error)
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 5, Start: 20, Length: 4, Severity: Error)
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 2, Start: 20, Length: 4, Severity: Error)
# too many failures !
# badcode:
greetings = "Hello"
show label "" a1b1 with greetings

### verifying challenge No. 001
# compile ok
# judge_decision: True

### verifying challenge No. 002
# compile ok
# judge_decision: True

### verifying challenge No. 003
# compile ok
# judge_decision: True

### verifying challenge No. 004
Compilation Failed!
Error: Found identifier but expected 'upload', 'table', tile-type or 'chart'. (Line: 4, Start: 5, Length: 1, Severity: Error)
Compilation Failed!
Error: Found identifier but expected 'upload', 'table', tile-type or 'chart'. (Line: 4, Start: 5, Length: 1, Severity: Error)
Compilation Failed!
Error: Found 